In [ ]:
# Dense Optical Flow using cv2 with Farneback method
# Farneback optical flow via webcam, 2 windows (arrows, color)
# press 'q' to exit

In [11]:
import numpy as np
import cv2
import time

# arrows showing how much motion in that image frame
def draw_flow(img, flow, step=16):

  h, w = img.shape[:2]
  y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
  fx, fy = flow[y, x].T

  lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
  lines = np.int32(lines + 0.5)

  img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

  for (x1, y1), (_x2, _y2) in lines:
    cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)
  
  return img_bgr

# draw a color image to see what is actually changign in the image

def draw_hsv(flow):

  h, w = flow.shape[:2]
  fx, fy = flow[:,:,0], flow[:,:,1]

  ang = np.arctan2(fy, fx) + np.pi
  v = np.sqrt(fx*fx + fy*fy)

  hsv = np.zeros((h, w, 3), np.uint8)
  hsv[...,0] = ang*(180/np.pi/2)
  hsv[...,1] = 255
  hsv[...,2] = np.minimum(v*4, 255)
  bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

  return bgr

cap = cv2.VideoCapture(0)
# video_path = "videos/ConstantScrolling.mp4" 
# cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

while True:
  suc, img = cap.read()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  start = time.time()

  # compute dense optical flow, store in in the variable
  # pass prev frame and currect frame to compare
  flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

  prevgray = gray

  end = time.time()

  fps = 1 / (end-start)

  print(f"{fps:.2f} FPS")

  cv2.imshow('flow', draw_flow(gray, flow))
  cv2.imshow('flow HSV', draw_hsv(flow))

  key = cv2.waitKey(30) & 0xFF
  if key == 27:  # ESC key to exit.
    break

  # key = cv2.waitKey(5)

  # if key == ord('q'):
  #   break

cap.release()
cv2.destroyAllWindows()


23.24 FPS
31.00 FPS
36.65 FPS
33.66 FPS
32.63 FPS
30.86 FPS
25.02 FPS
30.71 FPS
33.57 FPS
29.78 FPS
31.01 FPS
25.62 FPS
27.70 FPS
28.54 FPS
33.76 FPS
24.24 FPS
31.04 FPS
29.39 FPS
27.94 FPS
31.46 FPS
30.51 FPS
26.03 FPS
29.48 FPS
38.99 FPS
23.93 FPS
28.18 FPS
35.17 FPS
30.95 FPS
28.66 FPS
28.25 FPS
28.89 FPS
25.09 FPS
31.90 FPS
35.54 FPS
26.13 FPS
29.92 FPS
30.03 FPS
31.41 FPS
28.10 FPS
25.17 FPS
27.88 FPS
26.16 FPS
31.93 FPS
24.73 FPS
28.39 FPS
25.69 FPS
23.36 FPS
31.60 FPS
30.91 FPS
25.34 FPS
31.39 FPS
29.47 FPS
25.17 FPS
24.96 FPS
34.45 FPS
26.51 FPS
31.78 FPS
29.35 FPS
31.66 FPS
25.54 FPS
32.28 FPS
30.24 FPS
33.27 FPS
29.00 FPS
31.87 FPS
25.47 FPS
33.46 FPS
23.14 FPS
23.81 FPS
25.97 FPS
33.78 FPS
31.38 FPS
26.40 FPS
24.11 FPS
24.67 FPS
29.25 FPS
28.96 FPS
24.60 FPS
25.73 FPS
34.93 FPS
22.56 FPS
31.32 FPS
27.62 FPS
28.68 FPS
34.53 FPS
26.57 FPS
31.33 FPS
31.15 FPS
28.74 FPS
31.67 FPS
34.21 FPS
31.96 FPS
24.85 FPS
30.13 FPS
27.55 FPS
27.05 FPS
32.54 FPS
29.93 FPS
24.31 FPS
25.25 FPS


In [ ]:
# Sparse Optical Flow using cv2 with Lucas - Kanade method

In [8]:
import numpy as np
import cv2 as cv

# Open the default webcam and set the capture resolution.
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

# Create a resizable window and set its size to a larger display.
cv.namedWindow('Optical Flow Demo', cv.WINDOW_NORMAL)
cv.resizeWindow('Optical Flow Demo', 1280, 960)

# Parameters for ShiTomasi corner detection.
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Parameters for Lucas-Kanade optical flow.
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create an array of random colors.
color = np.random.randint(0, 255, (100, 3))

# Capture the first frame, flip it horizontally, and detect features.
ret, old_frame = cap.read()
if not ret:
    print("Error: Cannot capture frame from webcam.")
    cap.release()
    cv.destroyAllWindows()
    exit()

old_frame = cv.flip(old_frame, 1)
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing optical flow tracks.
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    # Flip frame for non-mirrored view.
    frame = cv.flip(frame, 1)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculate optical flow using Lucas-Kanade method.
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # If flow is found, select good points and draw the tracks.
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    else:
        # If tracking fails, you might want to reinitialize tracking.
        good_new = p0

    # Combine the current frame with the drawn tracks.
    img = cv.add(frame, mask)

    # Overlay demo instructions.
    cv.putText(img, "Optical Flow Demo - Press 'r' to reset, 'ESC' to exit", 
               (10, 60), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Display the image in the larger window.
    cv.imshow('Optical Flow Demo', img)

    key = cv.waitKey(30) & 0xFF
    if key == 27:  # ESC key to exit.
        break
    elif key == ord('r'):  # Reset tracking when 'r' is pressed.
        # Reinitialize tracking on the current frame.
        old_frame = frame.copy()
        old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
        p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        mask = np.zeros_like(old_frame)
        continue  # Skip updating the old frame below for this cycle.

    # Update for the next iteration.
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv.destroyAllWindows()